In [50]:
import os

In [51]:
%pwd

'f:\\'

In [52]:
os.chdir('../')

In [53]:
%pwd

'f:\\'

In [54]:
import pandas as pd

In [55]:
jet_data=pd.read_csv('artifacts/data_ingestion/Dataset/train_FD001.txt', sep=" ", header=None)
jet_data.columns = ["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5"
                    ,"sensor6","sensor7","sensor8","sensor9","sensor10","sensor11","sensor12","sensor13"
                    ,"sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
                    ,"sensor20","sensor21","sensor22","sensor23"]

jet_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/Dataset/train_FD001.txt'

In [ ]:
jet_data.info()

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [ ]:
from src.TimeSeriesProject .constants import *
from src.TimeSeriesProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
        root_dir=config.root_dir,
        STATUS_FILE=config.STATUS_FILE,
        unzip_data_dir = config.unzip_data_dir,
        all_schema=schema,
        )

        return data_validation_config

In [ ]:
import os
from src.TimeSeriesProject import logger

In [ ]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def RUL_calculator(self,df, df_max_cycles):
        max_cycle = df_max_cycles["cycle"]
        result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='id', right_index=True)
        result_frame["RUL"] = result_frame["max_cycle"] - result_frame["cycle"]
        result_frame.drop(['max_cycle'], axis=1, inplace=True)
        return result_frame


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir,sep=" ", header=None)
            data.columns = ["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5"
                    ,"sensor6","sensor7","sensor8","sensor9","sensor10","sensor11","sensor12","sensor13"
                    ,"sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
                    ,"sensor20","sensor21","sensor22","sensor23"]
            
            data.drop(['sensor22', 'sensor23'], axis=1, inplace=True)
            data.to_csv('artifacts/data_validation/output_data.csv', index=False)

            jet_id_and_rul = data.groupby(['id'])[["id" ,"cycle"]].max()
            jet_id_and_rul.set_index('id', inplace=True)
            jet_id_and_rul.to_csv('artifacts/data_validation/jet_id_and_rul.csv', index=False)


            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            data = self.RUL_calculator(data, jet_id_and_rul)
            data.to_csv('artifacts/data_validation/relevent_data.csv', index=False)
            

            return validation_status
        
        except Exception as e:
            raise e

In [56]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
    df = pd.read_csv('artifacts/data_validation/output_data.csv')
    df_max_cycles = pd.read_csv('artifacts/data_validation/jet_id_and_rul.csv')
    data_validation.RUL_calculator(df,df_max_cycles)
    
    
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'